In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

----------- ----- Visual of Original Data ------------

In [ ]:
ticker = yf.Ticker('AMD')
dfAMD = ticker.history(period='5y')

In [ ]:
dates = dfAMD.index.to_numpy()
high_prices = dfAMD['High'].to_numpy()

Plot the Historical Data

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(dates, high_prices, label='AMD High Prices')
plt.title('AMD High Prices For Five Years')
plt.xlabel('Date')
plt.ylabel('High Price (USD)')
plt.grid(True)
plt.legend()
plt.show()

Split Data into Train and Test Sets

In [ ]:
train_size = int(len(high_prices) * 0.90)  
train = high_prices[:train_size]  
test = high_prices[train_size:]  

Split Dates for Train and Test Sets

In [ ]:
train_dates = dates[:train_size]
test_dates = dates[train_size:]

Print train and test sizes

In [ ]:
print(f"Training data size: {len(train)}")
print(f"Test data size: {len(test)}")

-------------------- Function to Fit ARIMA Model and Forecast --------------------

In [ ]:
def StartARIMAForecasting(Actual, P, D, Q):
    model = ARIMA(Actual, order=(P, D, Q))
    model_fit = model.fit()  
    print(model_fit.summary())
    prediction = model_fit.forecast()[0]
    return prediction

-------------------- Function to Gather Results --------------------

In [ ]:
def gather_results(actual, predictions, metrics):
    # Gather actual values, predictions, and performance metrics
    result = {
        "Actual Values": actual,
        "Predictions": predictions,
        "Performance Metrics": metrics
    }
    return result

-------------------- Initial Setup --------------------

In [ ]:
log_train = np.log(train)
log_test = np.log(test)

Actual list from the log-transformed training data

In [ ]:
Actual = [x for x in log_train]  
Predictions = []  

-------------------- Forecasting Using ARIMA Model --------------------

In [ ]:
for timepoint in range(len(log_test)):
    ActualValue = log_test[timepoint]
    
    # Forecast value using ARIMA
    Prediction = StartARIMAForecasting(Actual, 1, 1, 0)  # (P, D, Q) 
   
    Predictions.append(Prediction)
    Actual.append(ActualValue)

-------------------- Reverse the Log Transformation --------------------

In [ ]:
forecast = np.exp(Predictions)  # Reverse the log transformation

-------------------- Plot the Results --------------------

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test_dates, np.exp(log_test), label='Test Data', color='green')  # Actual test data (reversed log)
plt.plot(test_dates, forecast, label='Forecast', color='red',linestyle='--' )  # Forecasted values (reversed log)
plt.plot(train_dates, np.exp(log_train), label='Training Data (Original Scale)', color='blue', linestyle='--')
plt.xlabel('Dates')
plt.ylabel('Prices (USD)')
plt.legend()
plt.title("AMD Stock Price Forecast vs Actual (Log Transformed Data)")
plt.grid(True)
plt.show()

-------------------- Performance Metrics --------------------

In [ ]:
mae = mean_absolute_error(np.exp(log_test), forecast)  # Using original scale for metrics
mse = mean_squared_error(np.exp(log_test), forecast)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((np.exp(log_test) - forecast) / np.exp(log_test))) * 100

In [ ]:
metrics = {
    "MAE": mae,
    "MSE": mse,
    "RMSE": rmse,
    "MAPE": mape
}

-------------------- Gather and Display Results --------------------

In [ ]:
gathered_results = gather_results(np.exp(log_test), forecast, metrics )

Print gathered results

In [ ]:
print("Gathered Results:")
print("Actual Values: ", gathered_results["Actual Values"])
print("Predictions: ", gathered_results["Predictions"])
print("Performance Metrics: ", gathered_results["Performance Metrics"])
print("Log-transformed Training Data (log_train):")
print(log_train)

In [ ]:
df = pd.DataFrame({
    "Date": test_dates,
    "Actual": np.exp(log_test),  # Reverse the log for actual data
    "Predictions": forecast
})
#df.to_csv("forecast_results.csv", index=False)
#print("\nResults saved to 'forecast_results.csv'.")

In [ ]:
log_high_prices = np.log(high_prices)

In [ ]:
final_model = ARIMA(log_high_prices, order=(2, 1, 2))  
final_model_fit = final_model.fit()

Start with the last known data point for rolling forecast

In [ ]:
last_known_value = log_high_prices[-1]
predictions_100_days = []

Perform a rolling forecast for the next 100 days

In [ ]:
for step in range(100):
    model = ARIMA(np.concatenate([log_high_prices, np.array(predictions_100_days)]), order=(1, 1, 0))
    model_fit = model.fit()
    print(model_fit.summary())
    next_pred_log = model_fit.forecast(steps=1)
    predictions_100_days.append(next_pred_log[0])

In [ ]:
future_forecast = np.exp(predictions_100_days)

In [ ]:
last_date = dates[-1]
future_dates = pd.date_range(last_date, periods=101, freq='B')[1:]
future_dates = np.array(future_dates)

-------------------- Plotting the 100-Day Forecast --------------------

In [ ]:
plt.figure(figsize=(12, 6))

Plot the historical data

In [ ]:
plt.plot(dates, high_prices, label='Historical Data', color='blue')

In [ ]:
plt.plot(future_dates, future_forecast, label='Future Predictions (Next 100 Days)', color='purple', linestyle='--')

In [ ]:
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.title('AMD Stock Price Prediction for the Next 100 Days (Rolling Forecast)')
plt.legend()
plt.grid(True)
plt.show()